In [ ]:
import sys
sys.path.append('../')

In [1]:
import os
import pandas as pd
from classes.embeddings import OpenaiAdaEmbedding, BertEmbedding
from tqdm import tqdm

In [2]:
api_key = os.environ['OPENAI_API_KEY']
openai_embedding = OpenaiAdaEmbedding(api_key)

In [2]:
api_key = os.environ['OPENAI_API_KEY']

openai_embedding = OpenaiAdaEmbedding(api_key)

df = pd.read_csv('../data/2023_articles/may/2023-05-16.csv')
df['text'].fillna("", inplace=True)

# Apply the get_embedding function to each row and create a new 'embedding' column
df['embedding'] = df['text'].apply(lambda x: openai_embedding.get_embedding(x))

# Save the DataFrame to a CSV file
df.to_csv('embedded.csv', index=False)


In [ ]:
path = '../data/2023_articles_en'
for folder in tqdm(os.listdir(path)):
    for filename in os.listdir(f'{path}/{folder}'):
        if filename.endswith(".csv"):
            df = pd.read_csv(f'{path}/{folder}/{filename}')
            # apply embeddings except rows without text
            df['ada_embedding'] = df['text'].apply(lambda x: openai_embedding.get_embedding(x) if x else None)
            df.to_csv(f'{path}/{folder}/{filename}_embedded.csv', index=False)

In [ ]:
# save all dataframes to one csv

full_df = pd.DataFrame()

for folder in tqdm(os.listdir(path)):
    for filename in os.listdir(f'{path}/{folder}'):
        if filename.endswith(".csv"):
            df = pd.read_csv(f'{path}/{folder}/{filename}')
            # create date column
            df['date'] = filename.split('.')[0]
            df['date'] = pd.to_datetime(df['date'], format='%Y-%m-%d')
            full_df = full_df.append(df)

In [26]:
# filter out rows without text
full_df = full_df[full_df['text'] != 'None']
full_df = full_df[['headline', 'keywords', 'categories', 'text', 'places', 'date', 'high_label', 'ada_embedding']]
full_df.to_csv('../data/articles_2023_en.csv', index=False)